# [editted]_Coding exercise 8

### Task 8.1
##### Create a bubble map for property crimes in Boston, 2019. Each bubble represents one property crime incident. Your map should look something like this example (Links to an external site.) online.

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import folium
import random
import matplotlib.pyplot as plt

path_homework8= "/Users/apple/Documents/BC/SOCY7700/compSoc/homework 8"
os.chdir(path_homework8)

crime2019_df = pd.read_csv('2019 Boston crime incident reports.csv')
crime2019_dropdu_df=crime2019_df.drop_duplicates(subset=['INCIDENT_NUMBER', 'OFFENSE_CODE', 'DISTRICT', 'Long', 'Lat', 'SHOOTING', 'OCCURRED_ON_DATE'])
cleaned_crime2019_df=crime2019_dropdu_df.dropna(how='any',subset=['INCIDENT_NUMBER', 'OFFENSE_CODE', 'DISTRICT', 'Long', 'Lat', 'SHOOTING', 'OCCURRED_ON_DATE'])
cleaned_crime2019_df.to_csv('cleaned_crime2019_data.csv', index=False)
cleaned_crime2019_df = pd.read_csv('cleaned_crime2019_data.csv')
cleaned1_crime2019_df = cleaned_crime2019_df[cleaned_crime2019_df.DISTRICT != 'External']
cleaned1_crime2019_df.to_csv('cleaned1_crime2019_data.csv', index=False)

cleaned1_crime2019_df['violent crime']=cleaned1_crime2019_df['OFFENSE_DESCRIPTION'].str.lower().str.contains('robbery|assault|rape')
cleaned1_crime2019_df['property crime']=cleaned1_crime2019_df['OFFENSE_DESCRIPTION'].str.lower().str.contains('burglary|larceny|theft|arson')

property_crime_location = cleaned1_crime2019_df[["property crime",'Long',"Lat"]]


property_crime_location_gdf = gpd.GeoDataFrame(property_crime_location, geometry=gpd.points_from_xy(property_crime_location['Long'], property_crime_location['Lat']))

property1_crime_location_gdf = property_crime_location_gdf[property_crime_location_gdf['Long'] < 0].reset_index()

clon, clat = property1_crime_location_gdf['Long'].mean(), property1_crime_location_gdf['Lat'].mean()
m = folium.Map(location=(clat, clon), zoom_start=12, width=800, height=800)

for i in range(0,len(property1_crime_location_gdf)):
    folium.CircleMarker(location=[property1_crime_location_gdf.iloc[i]['Lat'], property1_crime_location_gdf.iloc[i]['Long']],color='crimson',weight=1, radius=1).add_to(m)
display(m)

### Task 8.2
##### Create another bubble map for violent crimes in Boston, 2019. Use a different map design (marker color, tile style, etc).


In [ ]:
import os
import pandas as pd
import geopandas as gpd
import folium
import random
import matplotlib.pyplot as plt

path_homework8= "/Users/apple/Documents/BC/SOCY7700/compSoc/homework 8"
os.chdir(path_homework8)


cleaned1_crime2019_df = pd.read_csv('cleaned1_crime2019_data.csv')

cleaned1_crime2019_df['violent crime']=cleaned1_crime2019_df['OFFENSE_DESCRIPTION'].str.lower().str.contains('robbery|assault|rape')

violent_crime_location = cleaned1_crime2019_df[["violent crime",'Long',"Lat"]]

violent_crime_location_gdf = gpd.GeoDataFrame(violent_crime_location, geometry=gpd.points_from_xy(violent_crime_location['Long'], violent_crime_location['Lat']))

violent1_crime_location_gdf = violent_crime_location_gdf[violent_crime_location_gdf['Long'] < 0].reset_index()

clon, clat = violent1_crime_location_gdf['Long'].mean(), violent1_crime_location_gdf['Lat'].mean()
m = folium.Map(location=(clat, clon), zoom_start=12, width=800, height=800, tiles="Cartodb Positron")
for i in range(0,len(violent1_crime_location_gdf)):
    folium.CircleMarker(location=[violent1_crime_location_gdf.iloc[i]['Lat'], violent1_crime_location_gdf.iloc[i]['Long']],color='green',weight=1, radius=1).add_to(m)
display(m)


### Task 8.3
##### Calculate zoning district-level violent vs. property crime counts. Use boston's zoning map from here (Links to an external site.) to map individual crime incidents to zoning district. Create a bubble map for these two sets of counts. There should be two sets of bubbles in your map. One set represents district-level violent crime counts, and the other set district-level property crime counts. These two bubble sets should be colored differently, but the size of the bubble for both sets should indicate the relative size of the crime count. Use each district’s centroid location to place their bubbles on the map. [Hint: in geopandas,  `gdf[“geometry”].centroid` returns the centroid point of the corresponding geometry.]

In [ ]:
pip install shapely

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import folium

import random
import matplotlib.pyplot as plt
import rtree
import shapely


path_homework8= "/Users/apple/Documents/BC/SOCY7700/compSoc/homework 8"
os.chdir(path_homework8)
boston_gdf = gpd.read_file('Boston_Zoning_Subdistricts.geojson')
cleaned1_crime2019_df = pd.read_csv('cleaned1_crime2019_data.csv')

cleaned1_crime2019_df['offense_type'] = None
cleaned1_crime2019_df['offense_type'][cleaned1_crime2019_df['OFFENSE_DESCRIPTION'].str.lower().str.contains('robbery|assault|rape')] = 'violent crime'
cleaned1_crime2019_df['offense_type'][cleaned1_crime2019_df['OFFENSE_DESCRIPTION'].str.lower().str.contains('burglary|larceny|theft|arson')] = 'property crime'


crime2019_gdf = gpd.GeoDataFrame(cleaned1_crime2019_df, geometry=gpd.points_from_xy(cleaned1_crime2019_df['Long'], cleaned1_crime2019_df['Lat']))
crime2019_with_zone_gdf = gpd.sjoin(crime2019_gdf, boston_gdf, predicate="within", how='left')
crime2019_with_zone_gdf_1 = crime2019_with_zone_gdf[crime2019_with_zone_gdf['Long'] < 0].reset_index()

offense_data_district = crime2019_with_zone_gdf_1.groupby(['DISTRICT_left','offense_type']).agg({'Long': 'count','Lat': 'count'}).reset_index()
offense_data_district

clon, clat = offense_data_district['Long'].mean(), offense_data_district['Lat'].mean()
m = folium.Map(location=(clat, clon), zoom_start=12, width=800, height=800)

for i in range(len(offense_data_district)):
    crime_index = random.randrange(0, len(offense_data_district))
    crime_long, crime_lat = offense_data_district.loc[crime_index, 'Long'], offense_data_district.loc[crime_index, 'Lat']
    offense_type= offense_data_district.loc[crime_index, 'offense_type']
    if offense_type == 'violent crime':
        folium.CircleMarker((crime_lat, crime_long), opacity=0.7, fill_color='red', weight=2, radius=20).add_to(m)
    else:
        folium.CircleMarker((crime_lat, crime_long), opacity=0.7, fill_color='blue', weight=2, radius=10).add_to(m)
display(m)